In [ ]:
!git clone https://github.com/mim-solutions/bert_for_longer_texts.git

Cloning into 'bert_for_longer_texts'...
remote: Enumerating objects: 676, done.
remote: Counting objects: 100% (274/274), done.
remote: Compressing objects: 100% (148/148), done.
remote: Total 676 (delta 167), reused 187 (delta 122), pack-reused 402
Receiving objects: 100% (676/676), 4.54 MiB | 22.92 MiB/s, done.
Resolving deltas: 100% (354/354), done.


In [ ]:
%%capture
!pip install datasets transformers accelerate

In [ ]:
from google.colab import drive
from datasets import load_dataset, load_metric

path_to_data = "/content/drive/MyDrive/Data/merged_data.csv"

drive.mount("/content/drive/")

full_dataset = load_dataset("csv", data_files = path_to_data, split="train", trust_remote_code=True)
dataset = full_dataset.train_test_split(test_size=0.1)
print(dataset["train"][0])

Mounted at /content/drive/


Generating train split: 0 examples [00:00, ? examples/s]

{'user_id': 1043741637275312128, 'post_text': 'I’ve officially been in Chico for 4 years. Double the intended amount of time. One year to go which is wild. Omg The Undoing was so good but god men really ain’t ever shit. There were 2 in today’s!!! The universe is balanced https://t.co/Uko5HktNdM. @robbieheart_ @ashar1lh I feel attacked!!! This is my life lolol. @Bpritchett25 Like next month? I’m broke as hell right now. You down to go on a weekday?. My ballot is coming today! Keep and eye out for yours and send it in ASAP!. @EHurlocker9 It was wild 😂. @Emmmmmilyyyyyy I still haven’t gotten word and I’m supposed to fly out Monday 😭 I’m stressing. Are people still showing up to your work or are they refusing to come? Lol. 😞. Being in an uber with a mask is so fucking weird. I hate 2020.. @Emmmmmilyyyyyy ITS SO GOOD. I agreed to babysit tomorrow and I’m regretting it soooooooooooooooo much. @Emmmmmilyyyyyy I love that 😭💕 I need to watch Billy on the street!. I’m so tired omg omg omg and I 

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['user_id', 'post_text', 'label'],
        num_rows: 1010
    })
    test: Dataset({
        features: ['user_id', 'post_text', 'label'],
        num_rows: 113
    })
})

In [ ]:
X_train = dataset["train"]["post_text"]
y_train = dataset["train"]["label"]
X_test = dataset["test"]["post_text"]
y_test = dataset["test"]["label"]

In [ ]:
%cd /content/bert_for_longer_texts/
from bert_for_longer_texts.belt_nlp.bert_classifier_with_pooling import BertClassifierWithPooling

MODEL_PARAMS = {
    "num_labels": 2,
    "batch_size": 4,
    "learning_rate": 3e-5,
    "epochs": 3,
    "chunk_size": 510,
    "stride": 510,
    "minimal_chunk_length": 510,
    "maximal_text_length": 510 * 4,
    "pooling_strategy": "mean",
    "device": "cuda",
    "many_gpus": True,
}
model = BertClassifierWithPooling(**MODEL_PARAMS)

/content/bert_for_longer_texts


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
model.fit(X_train, y_train, epochs=3)

In [ ]:
type(y_test)

list

In [ ]:
model.save("/content/drive/MyDrive/longBERT/")

In [ ]:
model = model.load("/content/drive/MyDrive/longBERT")

In [ ]:
X_test[0]

'Germany bans armed neo-Nazi group, seizes weapons https://t.co/PlZYj8iDJo. Fines for ultra-Orthodox schools that violated lockdown canceled – report https://t.co/kFEzxgMilT. Germany charges 11 with far-right plot to target Muslims, overthrow government https://t.co/3BkVGldC5U. Israel’s Oil Refineries, UAE’s Mazrui set up joint venture for polymer imports https://t.co/CA6jnBfVrU. Turkey’s Erdogan urges dialogue with EU amid tensions over gas exploration https://t.co/2NxmNrhv4i. Argentines mourn Maradona, ‘the most human of gods’ https://t.co/AoqcR5tIdV. Pope plans pilgrimage visit to Iraq area of Abraham’s birthplace https://t.co/tC662ry4qz. Israel’s first fossil fuel-free stock index aims to put investors in the green https://t.co/1VrLeEYeys. Gantz welcomes renewed Palestinian security ties: ‘I made great efforts’ https://t.co/EmEL2zHh1D via @timesofisrael. Top Democratic senator opposes reentering Iran deal under previous terms https://t.co/mpAauMGgJH. Moderna’s top medical officer s

In [ ]:
import numpy as np
classes = model.predict(X_test).cpu().numpy()
accurate = sum(classes == np.array(y_test))
accuracy = accurate / len(y_test)

print(f"Test accuracy: {accuracy}")

Test accuracy: 0.9292035398230089


In [ ]:
text = "Just had the most amazing cup of coffee! ☕️ Starting the day with good vibes. Got a promotion at work! Hard work pays off. Celebrating with friends tonight! Just saw the most beautiful sunset at the beach. Feeling grateful for life's little moments. Finished reading a fantastic book. Highly recommend it! Can't believe it's been a year since I started my fitness journey. Feeling stronger and healthier every day. Spent the weekend hiking and exploring nature. Nothing beats fresh air and beautiful views. Met up with an old friend today. Catching up was just what I needed. Tried a new recipe today and it turned out delicious! Cooking is becoming my new favorite hobby. Spontaneous road trip! Sometimes the best adventures are unplanned. Finally mastered that yoga pose I've been struggling with. Persistence pays off! Had a relaxing spa day. Self-care is so important. Watching my favorite movie for the hundredth time and it still makes me happy. Adopted a puppy! My heart is so full of love. Received a thoughtful gift from a loved one today. Feeling so blessed. Spent the day at a theme park and had an absolute blast. Thrill rides are my jam! Attended a live concert and it was incredible. Music really is the best therapy. Got tickets to my favorite band's concert next month. Can't wait! Successfully completed a big project at work. Feeling accomplished and proud of my team. Gardening all day today. There's something so therapeutic about getting your hands dirty and nurturing plants. Had a picnic in the park with my family. So much love and laughter. Enjoying a lazy Sunday with a good book and some tea. Perfect way to recharge for the week ahead. Learned a new skill today. Always feels good to expand my horizons. Spent the day baking with my grandma. Cherishing these precious moments. Loving the new season of my favorite TV show. Can't stop binge-watching! Hosted a dinner party and it was a huge success. Great food, great company. Random act of kindness: paid for the person behind me in the coffee shop. Made my day to see their smile. Finally took that dance class I've been talking about. So much fun! Went stargazing last night. The universe is so vast and beautiful. Celebrating small victories today. Every step forward counts. Visited a museum and got lost in art and history. Feeling inspired. Had a fun game night with friends. Laughter truly is the best medicine. Loving life more and more every day! 🌞 Just had the best coffee ever at the new cafe downtown. Feeling so productive this week! Got a promotion at work—hard work pays off! Spent the weekend hiking with friends, nature is so healing. Trying out a new recipe tonight, wish me luck! Family game night was a blast, we laughed so much! Just finished a great book, can’t wait to start the next one. The weather has been perfect for morning runs, feeling energized! Celebrated a friend's birthday, so much fun! Attended an amazing concert, the energy was incredible. Volunteered at the local shelter today, so fulfilling. Just booked a spontaneous trip for next month, can't wait! Loving the new yoga class, feeling so zen. Spent the afternoon at the beach, it was so relaxing. Reconnected with an old friend, it was like no time had passed. Enjoying a quiet evening with a good movie. Started a new hobby, feeling so creative! The garden is blooming beautifully, spring is my favorite. Had a fantastic dinner with family, so grateful for them. Feeling accomplished after finishing a big project at work. Trying out some new workout routines, loving the challenge. Enjoying a cozy night in with a cup of tea. Got a new pet, they're bringing so much joy! Went to a local farmer’s market, everything was so fresh and delicious. Celebrating small wins today, feeling great! So proud of my team at work, we’re doing amazing things. Life is good!"

In [ ]:
model.predict([text]).cpu().numpy()

array([1])